# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 9 2022 9:15AM,239219,19,ZYM060922,"Zymergen, Inc.",Released
1,Jun 9 2022 9:14AM,239218,18,EYE42944,Eye Pharma Inc,Released
2,Jun 9 2022 9:11AM,239215,20,ALU0007024,Alumier Labs Inc.,Released
3,Jun 9 2022 9:11AM,239215,20,ALUR002001029,Alumier Labs Inc.,Released
4,Jun 9 2022 9:11AM,239215,20,ALUR111936573,Alumier Labs Inc.,Released
5,Jun 9 2022 9:11AM,239215,20,ALUR119866291,Alumier Labs Inc.,Released
6,Jun 9 2022 9:11AM,239215,20,ALUR371167994,Alumier Labs Inc.,Released
7,Jun 9 2022 9:11AM,239215,20,ALUR911732251,Alumier Labs Inc.,Released
8,Jun 9 2022 9:11AM,239215,20,ALUR930101234,Alumier Labs Inc.,Released
9,Jun 9 2022 9:11AM,239215,20,ALUR648051260,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
49,239215,Released,11
50,239216,Released,4
51,239217,Released,1
52,239218,Released,1
53,239219,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239215,NaN,NaN,11.0
239216,NaN,NaN,4.0
239217,NaN,NaN,1.0
239218,NaN,NaN,1.0
239219,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239083,0.0,1.0,0.0
239086,0.0,0.0,1.0
239107,0.0,0.0,1.0
239110,0.0,1.0,0.0
239122,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239083,0,1,0
239086,0,0,1
239107,0,0,1
239110,0,1,0
239122,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239083,0,1,0
1,239086,0,0,1
2,239107,0,0,1
3,239110,0,1,0
4,239122,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239083,,1,
1,239086,,,1
2,239107,,,1
3,239110,,1,
4,239122,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 9 2022 9:15AM,239219,19,"Zymergen, Inc."
1,Jun 9 2022 9:14AM,239218,18,Eye Pharma Inc
2,Jun 9 2022 9:11AM,239215,20,Alumier Labs Inc.
13,Jun 9 2022 9:11AM,239217,10,Eywa Pharma Inc.
14,Jun 9 2022 9:11AM,239216,10,ISDIN Corporation
18,Jun 9 2022 9:08AM,239210,10,ISDIN Corporation
80,Jun 9 2022 9:08AM,239213,19,"GUSA Granules USA, Inc."
81,Jun 9 2022 9:07AM,239214,19,"Bi-Coastal Pharma International, LLC"
82,Jun 9 2022 9:06AM,239212,19,"Bi-Coastal Pharma International, LLC"
83,Jun 9 2022 9:05AM,239206,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 9 2022 9:15AM,239219,19,"Zymergen, Inc.",,,1
1,Jun 9 2022 9:14AM,239218,18,Eye Pharma Inc,,,1
2,Jun 9 2022 9:11AM,239215,20,Alumier Labs Inc.,,,11
3,Jun 9 2022 9:11AM,239217,10,Eywa Pharma Inc.,,,1
4,Jun 9 2022 9:11AM,239216,10,ISDIN Corporation,,,4
5,Jun 9 2022 9:08AM,239210,10,ISDIN Corporation,,,62
6,Jun 9 2022 9:08AM,239213,19,"GUSA Granules USA, Inc.",,,1
7,Jun 9 2022 9:07AM,239214,19,"Bi-Coastal Pharma International, LLC",,,1
8,Jun 9 2022 9:06AM,239212,19,"Bi-Coastal Pharma International, LLC",,,1
9,Jun 9 2022 9:05AM,239206,10,ISDIN Corporation,,,22


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 9:15AM,239219,19,"Zymergen, Inc.",1,,
1,Jun 9 2022 9:14AM,239218,18,Eye Pharma Inc,1,,
2,Jun 9 2022 9:11AM,239215,20,Alumier Labs Inc.,11,,
3,Jun 9 2022 9:11AM,239217,10,Eywa Pharma Inc.,1,,
4,Jun 9 2022 9:11AM,239216,10,ISDIN Corporation,4,,
5,Jun 9 2022 9:08AM,239210,10,ISDIN Corporation,62,,
6,Jun 9 2022 9:08AM,239213,19,"GUSA Granules USA, Inc.",1,,
7,Jun 9 2022 9:07AM,239214,19,"Bi-Coastal Pharma International, LLC",1,,
8,Jun 9 2022 9:06AM,239212,19,"Bi-Coastal Pharma International, LLC",1,,
9,Jun 9 2022 9:05AM,239206,10,ISDIN Corporation,22,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 9:15AM,239219,19,"Zymergen, Inc.",1,,
1,Jun 9 2022 9:14AM,239218,18,Eye Pharma Inc,1,,
2,Jun 9 2022 9:11AM,239215,20,Alumier Labs Inc.,11,,
3,Jun 9 2022 9:11AM,239217,10,Eywa Pharma Inc.,1,,
4,Jun 9 2022 9:11AM,239216,10,ISDIN Corporation,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 9:15AM,239219,19,"Zymergen, Inc.",1.0,NaN,NaN
1,Jun 9 2022 9:14AM,239218,18,Eye Pharma Inc,1.0,NaN,NaN
2,Jun 9 2022 9:11AM,239215,20,Alumier Labs Inc.,11.0,NaN,NaN
3,Jun 9 2022 9:11AM,239217,10,Eywa Pharma Inc.,1.0,NaN,NaN
4,Jun 9 2022 9:11AM,239216,10,ISDIN Corporation,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 9:15AM,239219,19,"Zymergen, Inc.",1.0,0.0,0.0
1,Jun 9 2022 9:14AM,239218,18,Eye Pharma Inc,1.0,0.0,0.0
2,Jun 9 2022 9:11AM,239215,20,Alumier Labs Inc.,11.0,0.0,0.0
3,Jun 9 2022 9:11AM,239217,10,Eywa Pharma Inc.,1.0,0.0,0.0
4,Jun 9 2022 9:11AM,239216,10,ISDIN Corporation,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4066173,186.0,1.0,26.0
102,239086,1.0,0.0,0.0
12,478290,0.0,2.0,0.0
16,956566,2.0,2.0,0.0
18,956703,4.0,0.0,0.0
19,2391889,39.0,26.0,28.0
20,717473,31.0,11.0,4.0
21,478313,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4066173,186.0,1.0,26.0
1,102,239086,1.0,0.0,0.0
2,12,478290,0.0,2.0,0.0
3,16,956566,2.0,2.0,0.0
4,18,956703,4.0,0.0,0.0
5,19,2391889,39.0,26.0,28.0
6,20,717473,31.0,11.0,4.0
7,21,478313,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,186.0,1.0,26.0
1,102,1.0,0.0,0.0
2,12,0.0,2.0,0.0
3,16,2.0,2.0,0.0
4,18,4.0,0.0,0.0
5,19,39.0,26.0,28.0
6,20,31.0,11.0,4.0
7,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,186.0
1,102,Released,1.0
2,12,Released,0.0
3,16,Released,2.0
4,18,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,16,18,19,20,21
Status,,,,,,,,
Completed,26.0,0.0,0.0,0.0,0.0,28.0,4.0,0.0
Executing,1.0,0.0,2.0,2.0,0.0,26.0,11.0,0.0
Released,186.0,1.0,0.0,2.0,4.0,39.0,31.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,16,18,19,20,21
0,Completed,26.0,0.0,0.0,0.0,0.0,28.0,4.0,0.0
1,Executing,1.0,0.0,2.0,2.0,0.0,26.0,11.0,0.0
2,Released,186.0,1.0,0.0,2.0,4.0,39.0,31.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,16,18,19,20,21
0,Completed,26.0,0.0,0.0,0.0,0.0,28.0,4.0,0.0
1,Executing,1.0,0.0,2.0,2.0,0.0,26.0,11.0,0.0
2,Released,186.0,1.0,0.0,2.0,4.0,39.0,31.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()